In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install SimpleITK

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import cv2 as cv

import inspect
path = "/content/drive/Shareddrives/IA DL_project/ML IA/IMAGE_PROCESSING_PIPELINE"

if path not in sys.path:
  sys.path.append(path)

import luna_module
from luna_module import *

# List all function names in the luna_module
function_names = [name for name, obj in inspect.getmembers(luna_module) if inspect.isfunction(obj)]
print(function_names)

['annotations_by_uid', 'binarize_lung', 'binarize_lung_3d', 'binary_closing', 'binary_dilation', 'binary_erosion', 'binary_fill_holes', 'binary_opening', 'center_of_mass', 'clear_border', 'convert_annotation_df', 'convert_annotation_df_with_uid', 'create_3d_mask', 'create_annotations_mask', 'create_annotations_mask_by_uid', 'create_patch', 'debugger', 'draw_ellipsoid', 'find_by_uid', 'find_neighborhood_indices', 'find_neighborhood_indices_more_precise', 'get_slice_candidates', 'get_slice_candidates_old', 'get_slices', 'get_uids', 'img_by_uid', 'masked_annotations_by_uid', 'masked_annotations_with_info_by_uid', 'meta_by_uid', 'norm2float', 'norm2uint16', 'norm2uint8', 'normalize_intensity', 'plot_slices', 'process_slice_candidates', 'process_slices', 'remove_non_central_objects', 'sensitivity_score', 'sensitivity_score_more_precise', 'show_3_images', 'subset_by_uid', 'unwanted_object_filter']


# DATASETS

## FOR TRAINING

In [ ]:
LUNA_PATH = os.path.join(os.getcwd(), "drive", "Shareddrives", "IA DL_project", "ML IA", "LUNA16")

SUBSETS_PATH = os.path.join(LUNA_PATH, "candidates", "solidity_on_z") # path for subsets folder


In [ ]:
LUNA_PATH = os.path.join(os.getcwd(), "drive", "Shareddrives", "IA DL_project", "ML IA", "LUNA16")

ground_truth_annotations0 = pd.read_csv(os.path.join(LUNA_PATH, f"subset0_annotations_expanded.csv"), index_col="Unnamed: 0")
ground_truth_annotations1 = pd.read_csv(os.path.join(LUNA_PATH, f"subset1_annotations_expanded.csv"), index_col="Unnamed: 0")
ground_truth_annotations8 = pd.read_csv(os.path.join(LUNA_PATH, f"subset8_annotations_expanded.csv"), index_col="Unnamed: 0")

print(ground_truth_annotations0)
print(ground_truth_annotations1)
print(ground_truth_annotations8)

                                              seriesuid      coordX  \
23    1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896... -100.567944   
25    1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...   46.188539   
26    1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...   36.392044   
28    1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674...  136.434059   
86    1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048...  145.967465   
...                                                 ...         ...   
1125  1.3.6.1.4.1.14519.5.2.1.6279.6001.868211851413...  -22.951826   
1137  1.3.6.1.4.1.14519.5.2.1.6279.6001.898642529028...  -42.200674   
1138  1.3.6.1.4.1.14519.5.2.1.6279.6001.898642529028...   74.399625   
1141  1.3.6.1.4.1.14519.5.2.1.6279.6001.905371958588...  109.116637   
1180  1.3.6.1.4.1.14519.5.2.1.6279.6001.979083010707... -104.441975   

          coordY      coordZ  diameter_mm  \
23     67.260517 -231.816619     6.440879   
25     48.402806 -108.578632    13.596471   
26     76.77

In [ ]:
candidates0 = pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/candidates/solidity_on_z/subset0/subset0_candidates.csv", index_col="Unnamed: 0")
candidates1 = pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/candidates/solidity_on_z/subset1/subset1_candidates.csv", index_col="Unnamed: 0")

custom_features0 =  pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/feature_extractions/pretrained_gabors/subset0/custom_features.csv", index_col="Unnamed: 0")
custom_features1 =  pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/feature_extractions/pretrained_gabors/subset1/custom_features.csv", index_col="Unnamed: 0")

#hog_features0 =  pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/feature_extractions/pretrained_gabors/subset0/hog_features.csv", index_col="Unnamed: 0")
#hog_features1 =  pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/feature_extractions/pretrained_gabors/subset1/hog_features.csv", index_col="Unnamed: 0")

In [ ]:
print(len(candidates0))
print(len(custom_features0))
print(len(candidates0)-len(custom_features0))
print("-----------")
print(len(candidates1))
print(len(custom_features1))
print(len(candidates1)-len(custom_features1))


77509
77509
0
-----------
64355
64355
0


In [ ]:
print(candidates1)
print(candidates1[candidates1["Class"]==1])

                                               seriesuid    x    y    z  Class
0      1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760...  190  374   35    0.0
1      1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760...  167  382   37    0.0
2      1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760...  177  382   36    0.0
3      1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760...  184  382   35    0.0
4      1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760...  189  364   36    0.0
...                                                  ...  ...  ...  ...    ...
64350  1.3.6.1.4.1.14519.5.2.1.6279.6001.970264865033...  302  254  375    0.0
64351  1.3.6.1.4.1.14519.5.2.1.6279.6001.970264865033...  186  273  378    0.0
64352  1.3.6.1.4.1.14519.5.2.1.6279.6001.970264865033...  271  265  381    0.0
64353  1.3.6.1.4.1.14519.5.2.1.6279.6001.970264865033...  269  265  384    0.0
64354  1.3.6.1.4.1.14519.5.2.1.6279.6001.970264865033...  249  259  388    0.0

[64355 rows x 5 columns]
                          

In [ ]:
candidates0 = candidates0.sort_index()
candidates1 = candidates1.sort_index()

custom_features0 = custom_features0.sort_index()
custom_features1 = custom_features1.sort_index()

# hog_features0 = hog_features0.sort_index()
# hog_features1 = hog_features1.sort_index()

In [ ]:
# hog_features0["Class"] = custom_features0["Class"]
# hog_features1["Class"] = custom_features1["Class"]


custom_features0[["x", "y", "z"]] = candidates0[["x", "y", "z"]]
custom_features1[["x", "y", "z"]] = candidates1[["x", "y", "z"]]


custom_features0_positives = custom_features0[custom_features0["Class"]==1]
custom_features0_negatives = custom_features0.drop(custom_features0[custom_features0["Class"]==1].index)

custom_features1_positives = custom_features1[custom_features1["Class"]==1]
custom_features1_negatives = custom_features1.drop(custom_features1[custom_features1["Class"]==1].index)

# hog_features0_positives = hog_features0[hog_features0["Class"]==1]
# hog_features0_negatives = hog_features0.drop(hog_features0[hog_features0["Class"]==1].index)

# hog_features1_positives = hog_features1[hog_features1["Class"]==1]
# hog_features1_negatives = hog_features1.drop(hog_features1[hog_features1["Class"]==1].index)

# print(custom_features0_negatives)

print(custom_features1_positives)
# print(custom_features1_negatives)

       z_distance_from_boundary  z_distance_from_boundary_percent  area  \
419                         100                          0.357143     0   
420                         101                          0.360714     0   
2809                        141                          0.429878     0   
2820                        141                          0.429878     0   
3019                        133                          0.405488     0   
...                         ...                               ...   ...   
63481                       149                          0.450151     0   
63482                       150                          0.453172     0   
63500                       152                          0.459215     0   
63504                       155                          0.468278     0   
63910                       202                          0.466513     0   

       perimeter  compactness  eccentricity  major_axis_length  \
419          0.0          0.0    

In [ ]:
custom_features_all = pd.concat([custom_features0,custom_features1,], ignore_index=True)
# hog_features_all = pd.concat([hog_features0,hog_features1,], ignore_index=True)

custom_features_all_positives = pd.concat([custom_features0_positives,custom_features1_positives,], ignore_index=True)
custom_features_all_negatives = pd.concat([custom_features0_negatives,custom_features1_negatives,], ignore_index=True)

# hog_features_all_positives = pd.concat([hog_features0_positives,hog_features1_positives,], ignore_index=True)
# hog_features_all_negatives = pd.concat([hog_features0_negatives,hog_features1_negatives,], ignore_index=True)

In [ ]:
stripped_ground_truth_annotations0 = convert_annotation_df_with_uid(ground_truth_annotations0)
stripped_ground_truth_annotations1 = convert_annotation_df_with_uid(ground_truth_annotations1)

print(ground_truth_annotations0)
print(stripped_ground_truth_annotations0)
print("-------")
print(ground_truth_annotations1)
print(stripped_ground_truth_annotations1)

                                              seriesuid      coordX  \
23    1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896... -100.567944   
25    1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...   46.188539   
26    1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...   36.392044   
28    1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674...  136.434059   
86    1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048...  145.967465   
...                                                 ...         ...   
1125  1.3.6.1.4.1.14519.5.2.1.6279.6001.868211851413...  -22.951826   
1137  1.3.6.1.4.1.14519.5.2.1.6279.6001.898642529028...  -42.200674   
1138  1.3.6.1.4.1.14519.5.2.1.6279.6001.898642529028...   74.399625   
1141  1.3.6.1.4.1.14519.5.2.1.6279.6001.905371958588...  109.116637   
1180  1.3.6.1.4.1.14519.5.2.1.6279.6001.979083010707... -104.441975   

          coordY      coordZ  diameter_mm  \
23     67.260517 -231.816619     6.440879   
25     48.402806 -108.578632    13.596471   
26     76.77

### SECURITY CHECK FOR SENSITIVITY

In [ ]:
total_gt = 0
total_hits = 0
total_hits_count = 0

uids0 = set(stripped_ground_truth_annotations0["seriesuid"])
for i, uid in enumerate(uids0):

  uid = uid.replace(".csv", "")
  print(f"\n[START] - {uid} - case {i+1} out of {len(uids0)}")

  uid_candidates_df = candidates0[candidates0["seriesuid"] == uid]
  uid_annotations_df = annotations_by_uid(uid, stripped_ground_truth_annotations0)

  hit_i, hit_i_dict = find_neighborhood_indices_more_precise(uid_candidates_df, uid_annotations_df)

  # Counting nodules
  total_gt += len(hit_i_dict.keys())
  # Counting candidates that are in the vicinity of the nodule
  for i in hit_i_dict.values():
    total_hits_count += len(i)
    if len(i):
      total_hits += 1

  print(f"\n[CANDIDATES] - {uid}")
  print(uid_candidates_df.loc[hit_i].drop(columns=["seriesuid"]))
  print(f"\n[GROUND TRUTH] - {uid}")
  print(uid_annotations_df.drop(columns=["seriesuid"]))
  print(f"\n[HITS] - {uid}")
  print(hit_i_dict)


  print(f"\n[DONE] - {uid}")
  print("--------------------------------------")

print("--------------------------------------")
total_gt1 = 0
total_hits1 = 0
total_hits_count1 = 0

uids1 = set(stripped_ground_truth_annotations1["seriesuid"])
for i, uid in enumerate(uids1):

  uid = uid.replace(".csv", "")
  print(f"\n[START] - {uid} - case {i+1} out of {len(uids1)}")

  uid_candidates_df = candidates1[candidates1["seriesuid"] == uid]
  uid_annotations_df = annotations_by_uid(uid, stripped_ground_truth_annotations1)

  hit_i, hit_i_dict = find_neighborhood_indices_more_precise(uid_candidates_df, uid_annotations_df)

  # Counting nodules
  total_gt1 += len(hit_i_dict.keys())
  # Counting candidates that are in the vicinity of the nodule
  for i in hit_i_dict.values():
    total_hits_count1 += len(i)
    if len(i):
      total_hits1 += 1

  print(f"\n[CANDIDATES] - {uid}")
  print(uid_candidates_df.loc[hit_i].drop(columns=["seriesuid"]))
  print(f"\n[GROUND TRUTH] - {uid}")
  print(uid_annotations_df.drop(columns=["seriesuid"]))
  print(f"\n[HITS] - {uid}")
  print(hit_i_dict)


  print(f"\n[DONE] - {uid}")
  print("--------------------------------------")


print(f"\n[TOTAL HITS COUNT0]: {total_hits_count}")
print(f"[SENS0]: {total_hits/total_gt}")


print(f"\n[TOTAL HITS COUNT1]: {total_hits_count1}")
print(f"[SENS1]: {total_hits1/total_gt1}")


[START] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565 - case 1 out of 67

[CANDIDATES] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565
        x    y   z  Class
2988  457  229  41    1.0
2989  453  233  41    1.0
2990  456  233  41    1.0

[GROUND TRUTH] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565
      x    y   z  diam_x  diam_y  diam_z
86  457  229  41       9       9       4

[HITS] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565
{86: [2988, 2989, 2990]}

[DONE] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565
--------------------------------------

[START] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.249530219848512542668813996730 - case 2 out of 67

[CANDIDATES] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.249530219848512542668813996730
         x    y    z  Class
30407  441  310   96    1.0
30454  477  258  130    1.0
30802   73  264  246    1.0

[GROUND TRUTH] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.2

## FOR VALIDATION

In [ ]:
candidates8 = pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/candidates/solidity_on_z/subset8/subset8_candidates.csv", index_col="Unnamed: 0")

custom_features8 =  pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/feature_extractions/pretrained_gabors/subset8/custom_features.csv", index_col="Unnamed: 0")

# hog_features8 =  pd.read_csv("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/feature_extractions/pretrained_gabors/subset8/hog_features.csv", index_col="Unnamed: 0")

In [ ]:
stripped_ground_truth_annotations8 = convert_annotation_df_with_uid(ground_truth_annotations8)
stripped_ground_truth_annotations8

,seriesuid,x,y,z,diam_x,diam_y,diam_z
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,45,212,78,9,9,3
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,406,155,117,7,7,2
120,1.3.6.1.4.1.14519.5.2.1.6279.6001.131939324905...,125,372,83,19,19,7
121,1.3.6.1.4.1.14519.5.2.1.6279.6001.131939324905...,168,369,40,13,13,5
137,1.3.6.1.4.1.14519.5.2.1.6279.6001.137773550852...,419,197,193,30,30,18
...,...,...,...,...,...,...,...
1130,1.3.6.1.4.1.14519.5.2.1.6279.6001.882070241245...,364,310,109,26,26,7
1155,1.3.6.1.4.1.14519.5.2.1.6279.6001.931383239747...,295,338,44,8,8,5
1156,1.3.6.1.4.1.14519.5.2.1.6279.6001.931383239747...,154,286,80,8,8,5
1184,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,339,370,93,7,7,3


In [ ]:
candidates8 = candidates8.sort_index()

custom_features8 = custom_features8.sort_index()

# hog_features8 = hog_features8.sort_index()

In [ ]:
# hog_features8["Class"] = custom_features8["Class"]

custom_features8[["x", "y", "z"]] = candidates8[["x", "y", "z"]]

custom_features8_positives = custom_features8[custom_features8["Class"]==1]
custom_features8_negatives = custom_features8.drop(custom_features8[custom_features8["Class"]==1].index)

print(custom_features8_positives)
# hog_features8_positives = hog_features8[hog_features8["Class"]==1]
# hog_features8_negatives = hog_features8.drop(hog_features8[hog_features8["Class"]==1].index)

       z_distance_from_boundary  z_distance_from_boundary_percent  area  \
249                          78                          0.402062     0   
1038                         42                          0.308824     0   
1387                         54                          0.397059     0   
2227                        150                          0.437318     5   
2353                        138                          0.402332     8   
...                         ...                               ...   ...   
60454                       135                          0.473684     0   
60457                       137                          0.480702     0   
60458                       136                          0.477193     0   
60491                       139                          0.487719     0   
60495                       139                          0.487719     0   

       perimeter  compactness  eccentricity  major_axis_length  \
249     0.000000     0.000000    

In [ ]:
total_gt8 = 0
total_hits8 = 0
total_hits_count8 = 0

uids1 = set(stripped_ground_truth_annotations8["seriesuid"])
for i, uid in enumerate(uids1):

  uid = uid.replace(".csv", "")
  print(f"\n[START] - {uid} - case {i+1} out of {len(uids1)}")

  uid_candidates_df = candidates8[candidates8["seriesuid"] == uid]
  uid_annotations_df = annotations_by_uid(uid, stripped_ground_truth_annotations8)

  hit_i, hit_i_dict = find_neighborhood_indices_more_precise(uid_candidates_df, uid_annotations_df)

  # Counting nodules
  total_gt8 += len(hit_i_dict.keys())
  # Counting candidates that are in the vicinity of the nodule
  for i in hit_i_dict.values():
    total_hits_count8 += len(i)
    if len(i):
      total_hits8 += 1

  print(f"\n[CANDIDATES] - {uid}")
  print(uid_candidates_df.loc[hit_i].drop(columns=["seriesuid"]))
  print(f"\n[GROUND TRUTH] - {uid}")
  print(uid_annotations_df.drop(columns=["seriesuid"]))
  print(f"\n[HITS] - {uid}")
  print(hit_i_dict)


  print(f"\n[DONE] - {uid}")
  print("--------------------------------------")




print(f"\n[TOTAL HITS COUNT1]: {total_hits_count8}")
print(f"[SENS8]: {total_hits8/total_gt8}")


[START] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203 - case 1 out of 60

[CANDIDATES] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203
         x    y   z  Class
53600   89  124  65    1.0
53659  414  294  70    1.0
53693  404  290  71    1.0
53714  415  303  78    1.0

[GROUND TRUTH] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203
        x    y   z  diam_x  diam_y  diam_z
1064   89  124  66       8       8       3
1065  417  303  78      33      33      12

[HITS] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203
{1064: [53600], 1065: [53659, 53693, 53714]}

[DONE] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149938600000570588203
--------------------------------------

[START] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.346115813056769250958550383763 - case 2 out of 60

[CANDIDATES] - 1.3.6.1.4.1.14519.5.2.1.6279.6001.346115813056769250958550383763
         x    y    z  Class
47577  113  236  236    1.0

[GROUND TR

In [ ]:
raise Exception("STOOOOOOOOOOOOOOOOOOOOOOP")

Exception: STOOOOOOOOOOOOOOOOOOOOOOP

# EDA

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_distributions_compare_frequency(df1, df2):
    num_cols = df1.shape[1]
    rows = (num_cols // 3) + (1 if num_cols % 3 != 0 else 0)

    fig, axes = plt.subplots(rows, 6, figsize=(24, 4 * rows))
    axes = axes.flatten()

    colors1 = plt.cm.viridis(np.linspace(0, 1, num_cols))
    colors2 = plt.cm.plasma(np.linspace(0, 1, num_cols))

    for i, col in enumerate(df1.columns):
        ax1 = axes[2 * i]
        ax2 = axes[2 * i + 1]
        color1 = colors1[i % len(colors1)]
        color2 = colors2[i % len(colors2)]

        if df1[col].dtype == 'object':
            df1[col].value_counts(normalize=True).plot(kind='bar', ax=ax1, color=color1)
            df2[col].value_counts(normalize=True).plot(kind='bar', ax=ax2, color=color2)
        else:
            min_range = min(df1[col].min(), df2[col].min())
            max_range = max(df1[col].max(), df2[col].max())
            df1[col].plot(kind='hist', ax=ax1, color=color1, bins=20, density=True)
            df2[col].plot(kind='hist', ax=ax2, color=color2, bins=20, density=True)
            ax1.set_xlim(min_range, max_range)
            ax2.set_xlim(min_range, max_range)

        ax1.set_title(f'{col} (df1)')
        ax2.set_title(f'{col} (df2)')
        ax1.set_ylabel('Frequency')
        ax2.set_ylabel('Frequency')

    for i in range(2 * num_cols, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()


In [ ]:
plot_distributions_compare_frequency(custom_features0_positives, custom_features0_negatives)

In [ ]:
def plot_distributions_compare_count(df1, df2):
    num_cols = df1.shape[1]
    rows = (num_cols // 3) + (1 if num_cols % 3 != 0 else 0)

    fig, axes = plt.subplots(rows, 6, figsize=(24, 4 * rows))
    axes = axes.flatten()

    colors1 = plt.cm.viridis(np.linspace(0, 1, num_cols))
    colors2 = plt.cm.plasma(np.linspace(0, 1, num_cols))

    for i, col in enumerate(df1.columns):
        ax1 = axes[2 * i]
        ax2 = axes[2 * i + 1]
        color1 = colors1[i % len(colors1)]
        color2 = colors2[i % len(colors2)]

        if df1[col].dtype == 'object':
            df1[col].value_counts().plot(kind='bar', ax=ax1, color=color1)
            df2[col].value_counts().plot(kind='bar', ax=ax2, color=color2)
        else:
            min_range = min(df1[col].min(), df2[col].min())
            max_range = max(df1[col].max(), df2[col].max())
            df1[col].plot(kind='hist', ax=ax1, color=color1, bins=20)
            df2[col].plot(kind='hist', ax=ax2, color=color2, bins=20)
            ax1.set_xlim(min_range, max_range)
            ax2.set_xlim(min_range, max_range)

        ax1.set_title(f'{col} (df1)')
        ax2.set_title(f'{col} (df2)')
        ax1.set_ylabel('Count')
        ax2.set_ylabel('Count')

    for i in range(2 * num_cols, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
plot_distributions_compare_count(custom_features_all_positives, custom_features_all_negatives)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_distributions_compare_frequency_same_plot(df1, df2, cols=5):
    num_cols = df1.shape[1]
    rows = (num_cols // cols) + (1 if num_cols % cols != 0 else 0)

    fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows))
    axes = axes.flatten()

    colors1 = plt.cm.viridis(np.linspace(0, 1, num_cols))
    colors2 = plt.cm.plasma(np.linspace(0, 1, num_cols))

    for i, col in enumerate(df1.columns):
        ax = axes[i]
        color1 = colors1[i % len(colors1)]
        color2 = colors2[i % len(colors2)]

        if df1[col].dtype == 'object':
            df1[col].value_counts(normalize=True).plot(kind='bar', ax=ax, color=color1, alpha=0.7, position=1, width=0.4, label='df1')
            df2[col].value_counts(normalize=True).plot(kind='bar', ax=ax, color=color2, alpha=0.7, position=0, width=0.4, label='df2')
        else:
            min_range = min(df1[col].min(), df2[col].min())
            max_range = max(df1[col].max(), df2[col].max())
            df1[col].plot(kind='hist', ax=ax, color=color1, bins=20, density=True, alpha=0.7, label='df1')
            df2[col].plot(kind='hist', ax=ax, color=color2, bins=20, density=True, alpha=0.7, label='df2')
            ax.set_xlim(min_range, max_range)

        ax.set_title(col)
        ax.set_ylabel('Frequency')
        ax.legend()

    for i in range(num_cols, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
hog_features_all_positives
hog_features_all_negatives
plot_distributions_compare_frequency_same_plot(hog_features_all_positives, hog_features_all_negatives)


# CREATING ML MODELS

In [ ]:
!pip install imbalanced-learn
!pip install pymrmr


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler, NearMiss
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# Load data
df = custom_features_all

# Separate features and target
X = df.drop(columns=['Class', 'x', 'y', 'z'])
y = df['Class']

### mRMR class


In [ ]:
import pymrmr

# Custom transformer for MRMR feature selection
class MRMRFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, n_features_to_select=30):
        self.n_features_to_select = n_features_to_select
        self.selected_features = None

    def fit(self, X, y):
        # Ensure X is a DataFrame with string column names
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        X.columns = X.columns.astype(str)

        # Ensure y is a DataFrame with an appropriate name
        if not isinstance(y, pd.Series):
            y = pd.Series(y, name='target')
        else:
            y.name = 'target'

        # Concatenate X and y, and drop any NaN values
        df = pd.concat([X, y], axis=1).dropna()

        # Perform MRMR feature selection
        selected_features = pymrmr.mRMR(df, 'MIQ', self.n_features_to_select)
        self.selected_features = selected_features
        print(f"Selected features: {self.selected_features}")  # Debugging print
        return self

    def transform(self, X):
        # Convert X to DataFrame if it's not already
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        # Ensure X has the same column names as during fit
        X.columns = X.columns.astype(str)
        return X[self.selected_features]

In [ ]:
class MRMRFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, n_features_to_select=30):
        self.n_features_to_select = n_features_to_select
        self.selected_features = None

    def fit(self, X, y):
        # Ensure X is a DataFrame with string column names
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        X.columns = X.columns.astype(str)

        # Ensure y is a DataFrame with an appropriate name
        if not isinstance(y, pd.Series):
            y = pd.Series(y, name='target')
        else:
            y.name = 'target'

        # Concatenate X and y, and drop any NaN values
        df = pd.concat([X, y], axis=1).dropna()

        # Perform MRMR feature selection
        self.selected_features = pymrmr.mRMR(df, 'MIQ', self.n_features_to_select)
        return self

    def transform(self, X):
        # Convert X to DataFrame if it's not already
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        # Ensure X has the same column names as during fit
        X.columns = X.columns.astype(str)
        return X[self.selected_features]


## #1 attempt

1. ClusterCentroids: Used to handle class imbalance.
2. StandardScaler: Standardizes features by removing the mean and scaling to unit variance.
3. RandomForestClassifier: A robust model for classification tasks. class_weight='balanced' helps to handle class imbalance.
4. Threshold Tuning: Adjusting the threshold can help in controlling the balance between false positives and false negatives.
By following these steps, you should

In [ ]:
# Load data
df = custom_features_all

# Separate features and target
X = df.drop(columns=['Class'])
y = df['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Create a pipeline with ClusterCentroids, standard scaler, and random forest classifier
pipeline1 = Pipeline([
    ('cluster_centroids', ClusterCentroids(random_state=42)),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42, class_weight={0:1, 1:100}))
])

# Define parameter grid for GridSearchCV
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    #'classifier__max_depth': [None, 10, 20],
    'cluster_centroids__sampling_strategy': ['majority'],
    #'cluster_centroids__voting': ['hard', 'soft', 'auto']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline1, param_grid, cv=5, scoring='roc_auc', n_jobs=2, verbose=4)

# Fit the model
grid_search.fit(X_train, y_train)

# Print the best parameters and best score from GridSearchCV
print("Best parameters found: ", grid_search.best_params_)
print("Best ROC-AUC score: ", grid_search.best_score_)

# Train the final model on the full training set using the best parameters
pipeline1.set_params(**grid_search.best_params_)
pipeline1.fit(X_train, y_train)

# Evaluate on the test set
y_test_pred1 = pipeline1.predict(X_test)
print(classification_report(y_test, y_test_pred1))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred1)}")

# Optional: Adjust the probability threshold for predictions
y_test_proba1 = pipeline1.predict_proba(X_test)[:, 1]
threshold = 0.3  # Example threshold, adjust based on your needs
y_test_pred_adjusted1 = (y_test_proba1 >= threshold).astype(int)
print(classification_report(y_test, y_test_pred_adjusted1))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted1)}")


In [ ]:

y_test_pred = pipeline1.predict(X_test)
y_test_proba = pipeline1.predict_proba(X_test)[:, 1]
cm = confusion_matrix(y_test, y_test_pred)

print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred)}")
print(f"test len: {len(y_test_pred)}")
print(f"positives: {len(np.array(y_test_pred).nonzero()[0])}")
print(f"gt_positives: {len(np.array(y_test).nonzero()[0])}")
print(f"sensitivity: {cm[1,1]/sum(cm[1,:])}")
print(f"specificity: {cm[0,0]/sum(cm[0,:])}")
print("Confusion matrix")
print(cm)


print("----------")


threshold = 0.4 # Example threshold, adjust based on your needs
y_test_pred_adjusted1 = (y_test_proba >= threshold).astype(int)
cm = confusion_matrix(y_test, y_test_pred_adjusted1)

print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted1)}")
print(f"test len: {len(y_test_pred_adjusted1)}")
print(f"positives: {len(np.array(y_test_pred_adjusted1).nonzero()[0])}")
print(f"gt_positives: {len(np.array(y_test).nonzero()[0])}")
print(f"sensitivity: {cm[1,1]/sum(cm[1,:])}")
print(f"specificity: {cm[0,0]/sum(cm[0,:])}")

print("Confusion matrix")
print(cm)

## #2 attempt

1. Custom Class Weights: class_weights = {0: 1, 1: 10} assigns a higher weight (e.g., 10) to the minority class (1) compared to the majority class (0). You can adjust these weights based on your specific requirements.
2. RandomForestClassifier: The custom class weights are passed to the classifier.
3. Pipeline: Combines under-sampling, feature scaling, feature selection, and model training.

In [ ]:


# Load data
df = custom_features_all

# Separate features and target
X = df.drop(columns=['Class'])
y = df['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Define custom class weights (example: higher weight for the minority class)
class_weights = {0: 1, 1: 100}  # Adjust these values as needed

# Define the model with custom class weights
model = RandomForestClassifier(random_state=42, class_weight=class_weights)

# Perform RFE
rfe = RFE(estimator=LogisticRegression(), step=1)

# Create a pipeline with RandomUnderSampler, RFE, standard scaler, and random forest classifier
pipeline2 = Pipeline([
    ('under', RandomUnderSampler(random_state=42)),
    ('scaler', StandardScaler()),
    ('rfe', rfe),
    ('classifier', model)
])

# Define parameter grid for GridSearchCV
param_grid = {
    'classifier__n_estimators': [100],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'rfe__n_features_to_select': [30, 50, 100]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline2, param_grid, cv=5, scoring='roc_auc', n_jobs=2, verbose=4)

# Fit the model
grid_search.fit(X_train, y_train)

# Print the best parameters and best score from GridSearchCV
print("Best parameters found: ", grid_search.best_params_)
print("Best ROC-AUC score: ", grid_search.best_score_)

# Train the final model on the full training set using the best parameters
pipeline2.set_params(**grid_search.best_params_)
pipeline2.fit(X_train, y_train)

# Evaluate on the test set
y_test_pred2 = pipeline2.predict(X_test)
print(classification_report(y_test, y_test_pred2))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred2)}")

# Optional: Adjust the probability threshold for predictions
y_test_proba2 = pipeline2.predict_proba(X_test)[:, 1]
threshold = 0.1  # Example threshold, adjust based on your needs
y_test_pred_adjusted2 = (y_test_proba2 >= threshold).astype(int)
print(classification_report(y_test, y_test_pred_adjusted2))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted2)}")


In [ ]:
grid_search.best_params_

In [ ]:
from sklearn.metrics import confusion_matrix

y_test_pred = pipeline2.predict(X_test)
y_test_proba = pipeline2.predict_proba(X_test)[:, 1]
cm = confusion_matrix(y_test, y_test_pred)

print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred)}")
print(f"test len: {len(y_test_pred)}")
print(f"positives: {len(np.array(y_test_pred).nonzero()[0])}")
print(f"gt_positives: {len(np.array(y_test).nonzero()[0])}")
print(f"sensitivity: {cm[1,1]/sum(cm[1,:])}")
print(f"specificity: {cm[0,0]/sum(cm[0,:])}")
print("Confusion matrix")
print(cm)


print("----------")


threshold = 0.4 # Example threshold, adjust based on your needs
y_test_pred_adjusted1 = (y_test_proba >= threshold).astype(int)
cm = confusion_matrix(y_test, y_test_pred_adjusted1)

print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted1)}")
print(f"test len: {len(y_test_pred_adjusted1)}")
print(f"positives: {len(np.array(y_test_pred_adjusted1).nonzero()[0])}")
print(f"gt_positives: {len(np.array(y_test).nonzero()[0])}")
print(f"sensitivity: {cm[1,1]/sum(cm[1,:])}")
print(f"specificity: {cm[0,0]/sum(cm[0,:])}")

print("Confusion matrix")
print(cm)

### #2.1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_selection import RFE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Load data
df = custom_features_all

# Separate features and target
X = df.drop(columns=['Class'])
y = df['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Define custom class weights (example: higher weight for the minority class)
class_weights = {0: 1, 1: 100}  # Adjust these values as needed

# Define the model with custom class weights
model = RandomForestClassifier(random_state=42, class_weight=class_weights)

# Perform RFE
rfe = RFE(estimator=LogisticRegression(max_iter=1000, solver='saga'), n_features_to_select=30, step=1)

# Create a pipeline with RandomUnderSampler, RFE, standard scaler, and random forest classifier
pipeline = Pipeline([
    ('under', RandomUnderSampler(random_state=42)),
    ('scaler', StandardScaler()),
    ('rfe', rfe),
    ('classifier', model)
])

# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, val_index in skf.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    pipeline.fit(X_train_fold, y_train_fold)
    y_val_pred = pipeline.predict(X_val_fold)
    print(classification_report(y_val_fold, y_val_pred))
    print(f"ROC-AUC: {roc_auc_score(y_val_fold, y_val_pred)}")

# Train the final model on the full training set
pipeline.fit(X_train, y_train)

# Evaluate on the test set
y_test_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_test_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred)}")

# Optional: Adjust the probability threshold for predictions
y_test_proba = pipeline.predict_proba(X_test)[:, 1]
threshold = 0.1  # Example threshold, adjust based on your needs
y_test_pred_adjusted = (y_test_proba >= threshold).astype(int)
print(classification_report(y_test, y_test_pred_adjusted))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted)}")

In [ ]:
from sklearn.metrics import confusion_matrix


y_test_pred = pipeline.predict(X_test)
y_test_proba = pipeline.predict_proba(X_test)[:, 1]


print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred)}")
print(f"test len: {len(y_test_pred)}")
print(f"positives: {len(np.array(y_test_pred).nonzero()[0])}")
print(f"gt_positives: {len(np.array(y_test).nonzero()[0])}")

cm = confusion_matrix(y_test, y_test_pred)
print("Confusion matrix")
print(cm)

print("----------")

threshold = 0.3 # Example threshold, adjust based on your needs
y_test_pred_adjusted = (y_test_proba >= threshold).astype(int)
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted)}")


print(f"test len: {len(y_test_pred_adjusted)}")
print(f"positives: {len(np.array(y_test_pred_adjusted).nonzero()[0])}")
print(f"gt_positives: {len(np.array(y_test_pred_adjusted).nonzero()[0])}")

cm = confusion_matrix(y_test, y_test_pred_adjusted)
print("Confusion matrix")
print(cm)

### #2.2


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_selection import RFE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Load data
df = custom_features_all

# Separate features and target
X = df.drop(columns=['Class'])
y = df['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Define custom class weights (example: higher weight for the minority class)
class_weights = {0: 1, 1: 100}  # Adjust these values as needed

# Define the model with custom class weights
model = RandomForestClassifier(random_state=42, class_weight=class_weights)

# Perform RFE
rfe = RFE(estimator=LogisticRegression(max_iter=500, solver='saga'), n_features_to_select=100, step=1)

# Create a pipeline with RandomUnderSampler, RFE, standard scaler, and random forest classifier
pipeline = Pipeline([
    ('under', RandomUnderSampler(random_state=42)),
    ('scaler', StandardScaler()),
    ('rfe', rfe),
    ('classifier', model)
])

# Define parameter grid for GridSearchCV
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Print the best parameters and best score from GridSearchCV
print("Best parameters found: ", grid_search.best_params_)
print("Best ROC-AUC score: ", grid_search.best_score_)

# Train the final model on the full training set using the best parameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train, y_train)

# Evaluate on the test set
y_test_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_test_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred)}")

# Optional: Adjust the probability threshold for predictions
y_test_proba = pipeline.predict_proba(X_test)[:, 1]
threshold = 0.1  # Example threshold, adjust based on your needs
y_test_pred_adjusted = (y_test_proba >= threshold).astype(int)
print(classification_report(y_test, y_test_pred_adjusted))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted)}")


## #3 attempt

## #4 attempt

### PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score


X = df.drop(columns=['Class'])
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=96)  # Change 50 to the number of components you desire
X_pca = pca.fit_transform(X_scaled)

explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)
print(f'Explained variance by each component: {explained_variance_ratio}')
print(f'Cumulative explained variance: {cumulative_explained_variance}')

In [ ]:
# Determine the number of components needed for 95% variance
n_components_95 = np.where(cumulative_variance >= 0.95)[0][0]+1
print(f'Number of components to retain 95% variance: {n_components_95}')

plt.figure(figsize=(10, 5))
plt.plot(cumulative_explained_variance, marker='o')

# Draw a horizontal line at y=10
plt.axhline(y=0.95, color='coral', linestyle='--', linewidth=1)

# Draw a vertical line at x=2
plt.axvline(x=n_components_95, color='coral', linestyle='--', linewidth=1)

plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Principal Components')
plt.grid(True)
plt.show()

## #5 attempt


In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline

X = df.drop(columns=['Class'])
y = df['Class']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)



pipeline5 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=62)),
    ('rf', RandomForestClassifier(random_state=42))
])


# Initialize a model with hyperparameter grid
param_grid = {
    'rf__class_weight': [{0: 1, 1: 100}],  # Keeping the same class_weight as in the original model
    'rf__n_estimators': [100, 200],  # Example hyperparameters to tune
    'rf__max_depth': [None, 10, 20]  # Example hyperparameters to tune
}

# Initialize the model (without specifying class_weight here, as it will be part of the grid)
model = RandomForestClassifier()

# Perform stratified 5-fold cross-validation
stratified_kfold = StratifiedKFold(n_splits=5)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=pipeline5, param_grid=param_grid, cv=stratified_kfold, verbose=10, return_train_score=True)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best estimator
best_model = grid_search.best_estimator_

# Print the best model and its parameters
print(f"Best Model:\n{best_model}\n")

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best Parameters:\n{best_params}\n")


# Print the best parameters and best score from GridSearchCV
print("Best parameters found: ", grid_search.best_params_)
print("Best ROC-AUC score: ", grid_search.best_score_)

# Train the final model on the full training set using the best parameters
pipeline5.set_params(**grid_search.best_params_)
pipeline5.fit(X_train, y_train)





# # Get all the fitted models for each fold
# fitted_models = grid_search.cv_results_['params']

# # Print all the fitted models for each fold
# for i, params in enumerate(fitted_models):
#     print(f"Model for fold {i} parameters:\n{params}\n")


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
# Evaluate on the test set
y_test_pred5 = pipeline5.predict(X_test)
print(classification_report(y_test, y_test_pred5))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred5)}")

# Optional: Adjust the probability threshold for predictions
y_test_proba5 = pipeline5.predict_proba(X_test)[:, 1]
threshold = 0.1  # Example threshold, adjust based on your needs
y_test_pred_adjusted5 = (y_test_proba5 >= threshold).astype(int)
print(classification_report(y_test, y_test_pred_adjusted5))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted5)}")

In [ ]:
from sklearn.metrics import confusion_matrix

y_test_pred = pipeline5.predict(X_test)
y_test_proba = pipeline5.predict_proba(X_test)[:, 1]
cm = confusion_matrix(y_test, y_test_pred)

print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred)}")
print(f"test len: {len(y_test_pred)}")
print(f"positives: {len(np.array(y_test_pred).nonzero()[0])}")
print(f"gt_positives: {len(np.array(y_test).nonzero()[0])}")
print(f"sensitivity: {cm[1,1]/sum(cm[1,:])}")
print(f"specificity: {cm[0,0]/sum(cm[0,:])}")
print("Confusion matrix")
print(cm)


print("----------")


threshold = 0.4 # Example threshold, adjust based on your needs
y_test_pred_adjusted1 = (y_test_proba >= threshold).astype(int)
cm = confusion_matrix(y_test, y_test_pred_adjusted1)

print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted1)}")
print(f"test len: {len(y_test_pred_adjusted1)}")
print(f"positives: {len(np.array(y_test_pred_adjusted1).nonzero()[0])}")
print(f"gt_positives: {len(np.array(y_test).nonzero()[0])}")
print(f"sensitivity: {cm[1,1]/sum(cm[1,:])}")
print(f"specificity: {cm[0,0]/sum(cm[0,:])}")

print("Confusion matrix")
print(cm)

## #6 attempt

In [ ]:
rus = RandomUnderSampler(sampling_strategy='majority',)

X_resampled, y_resampled = rus.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)

## #7 attempt

In [ ]:
# Load data
df = custom_features_all

# Separate features and target
X = df.drop(columns=['Class', 'x', 'y', 'z'])
y = df['Class']

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_fscore_support
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import pymrmr

# Custom transformer for MRMR feature selection
class MRMRFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, n_features_to_select=30):
        self.n_features_to_select = n_features_to_select
        self.selected_features = None

    def fit(self, X, y):
        # Ensure X is a DataFrame with string column names
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        X.columns = X.columns.astype(str)

        # Ensure y is a DataFrame with an appropriate name
        if not isinstance(y, pd.Series):
            y = pd.Series(y, name='target')
        else:
            y.name = 'target'

        # Concatenate X and y, and drop any NaN values
        df = pd.concat([X, y], axis=1).dropna()

        # Perform MRMR feature selection
        selected_features = pymrmr.mRMR(df, 'MIQ', self.n_features_to_select)
        self.selected_features = selected_features
        print(f"Selected features: {self.selected_features}")  # Debugging print
        return self

    def transform(self, X):
        # Convert X to DataFrame if it's not already
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        # Ensure X has the same column names as during fit
        X.columns = X.columns.astype(str)
        return X[self.selected_features]

# Load data
df = custom_features_all

# Separate features and target
X = df.drop(columns=['Class', 'x', 'y', 'z'])
y = df['Class']

# Class Weights
class_weights = {
    0: 1,
    1: 400
}
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Define the RandomForest model with class weights
model_rf = RandomForestClassifier(
    class_weight=class_weights,
    random_state=42,
    bootstrap=True,
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=50
)
# Best param in GridSearch 'classifier__bootstrap': True, 'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50

# Create a pipeline with RandomUnderSampler, standard scaler, MRMR, and RandomForest classifier
pipeline_rf_gs= Pipeline([
    ('under', RandomUnderSampler(random_state=42)),
    ('scaler', StandardScaler()),
    ('mrmr', MRMRFeatureSelector(n_features_to_select=50)),
    ('classifier', model_rf)
])

# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, val_index in skf.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    pipeline_rf_gs.fit(X_train_fold, y_train_fold)
    y_val_pred = pipeline_rf_gs.predict(X_val_fold)
    print(classification_report(y_val_fold, y_val_pred))
    print(f"ROC-AUC: {roc_auc_score(y_val_fold, y_val_pred)}")

print("------------------------------")
# Train the final model on the full training set
pipeline_rf_gs.fit(X_train, y_train)

# Evaluate on the test set
y_test_pred = pipeline_rf_gs.predict(X_test)
print(classification_report(y_test, y_test_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred)}")

# Optional: Adjust the probability threshold for predictions
y_test_proba = pipeline_rf_gs.predict_proba(X_test)[:, 1]
threshold = 0.41  # Example threshold, adjust based on your needs
y_test_pred_adjusted = (y_test_proba >= threshold).astype(int)
print(classification_report(y_test, y_test_pred_adjusted))
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_pred_adjusted)}")

# Confusion Matrix and Analysis
conf_matrix = confusion_matrix(y_test, y_test_pred_adjusted)
tn, fp, fn, tp = conf_matrix.ravel()
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_test_pred_adjusted, average='binary')

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
X_train.columns

Index(['z_distance_from_boundary', 'z_distance_from_boundary_percent', 'area',
       'perimeter', 'compactness', 'eccentricity', 'major_axis_length',
       'minor_axis_length', 'solidity', 'extent',
       ...
       'gabor_features_max53', 'gabor_features_max54', 'gabor_features_max55',
       'gabor_features_max56', 'gabor_features_max61', 'gabor_features_max62',
       'gabor_features_max63', 'gabor_features_max64', 'gabor_features_max65',
       'gabor_features_max66'],
      dtype='object', length=315)